<a href="https://colab.research.google.com/github/hrnrhty/my-vae-nnabla/blob/step3/step3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 3 - オリジナルデータセットの作成

Step 1 では [MNIST](http://yann.lecun.com/exdb/mnist/) データセットを使って VAE (Variational Auto Encoder) の学習を行いました。続く Step 2 では、学習済みパラメータを読み込んで推論を実行しました。Step 2 で確認した通り、VAE は学習に用いたデータの特徴を学習し、学習データに共通して含まれる特徴を再現するような出力データを生成します。しかし、これがいったい何の役に立つのでしょうか？実は、この特性は「異常検知」に応用することができるのです。

異常とは、「正常時にはない特徴」と考えることができます。正常なデータのみを使ってトレーニングされた VAE は、正常データに共通する特徴しか再現できません。このため、推論時に異常が含まれるデータを入力すると、異常箇所が欠落したデータを出力します。つまり、入出力データを比べて差の多い部分が異常であると考えることができるのです。この特性を利用し、入力データと VAE 出力データとを比較することによって、差が大きい部分を異常箇所として可視化したり、その差が一定値以上の時に異常と判定するなど、異常検知アプリケーションに応用することができるのです。

本ステップ以降では、実際に VAE を異常検知に応用していきます。まずはじめに、本ステップではオリジナルのデータセットを作成してみましょう。

## Google Drive のマウント

作成したデータセットの永続化のため Google Drive をマウントします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 画像の読み込み

In [ ]:
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt

im = Image.open('drive/MyDrive/images/cardboard.jpg')
plt.imshow(im)
plt.show()

読み込んだ画像のサイズを確認してみましょう。下記のコードで、読み込んだ画像は幅 4032、高さ 3024 であることがわかります。

In [ ]:
print('(width, height) =', im.size)

In [ ]:
im_small = im.resize((im.width // 12, im.height // 12), resample=Image.LANCZOS)
plt.imshow(im_small)
plt.show()
print('(width, height) =', im_small.size)

In [ ]:
im_gray = im_small.convert('L')
plt.imshow(im_gray, cmap='gray')
plt.show()

In [ ]:
width = 28
height = 28
div_h = im_gray.width // width
div_v = im_gray.height // height

im_cropped = []
for v in range(div_v):
    for h in range(div_h):
        left = h * width
        upper = v * height
        right = (h + 1) * width
        lower = (v + 1) * height
        im_cropped.append(im_gray.crop((left, upper, right, lower)))
len(im_cropped)

In [ ]:
print(div_h, div_v)
from PIL import ImageDraw
draw = ImageDraw.Draw(im_small)
v, h = (2, 3)  # 3行目4列目傷 idx=27  
draw.rectangle((28*h, 28*v, 28*h+28, 28*v+28), fill=None, outline='red')
v, h = (6, 7)  # 7行目8列目インク idx=79
draw.rectangle((28*h, 28*v, 28*h+28, 28*v+28), fill=None, outline='blue')
plt.imshow(im_small)
plt.show()

In [ ]:
print(len(im_cropped))
im_ad_test = []
im_ad_test.append(im_cropped.pop(79))
im_ad_test.append(im_cropped.pop(27))

In [ ]:
plt.imshow(im_ad_test[0], cmap='gray')
plt.show()

In [ ]:
plt.imshow(im_ad_test[1], cmap='gray')
plt.show()

### TODO
NG 2 個 + OK 6 個、計 8 個 を性能検証用データに、残り 100 個を学習用データに分ける。
画像はグレースケール 28 x 28 で通し番号を付けて保存。
データセットは CSV 形式で、1行目はヘッダ。NNC と同じ形式が読み込めるようなので、NNC のマニュアルに従い CSV ファイルを作る。

In [ ]:
len(im_cropped)